# NamuyWamChat

En este cuaderno vamos a construir un chat en Namuy Wam (una lengua indigena en Colombia) vamos a integrar herramientas que nos permitan enriquecer el contexto de la conversación como: 

1. Vector Stores
2. Conversation Buffer Memory

In [1]:
from langchain.llms.bedrock import Bedrock

In [2]:
# invoke the LLM
llm_claude = Bedrock(
  model_id="anthropic.claude-v2",
  model_kwargs={'max_tokens_to_sample': 500},
  credentials_profile_name="default", 
  region_name="us-east-1"
)

In [22]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import JSONLoader
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import json

In [23]:
translations = None
with open('data/translations.json') as translations_file:
    translations = json.load(translations_file)
    translations = translations['translations']
print(translations)

[{'es': 'Levantar', 'gum': 'Kutrap'}, {'es': 'Levantarse', 'gum': 'Kia kutrap'}, {'es': 'Bañar', 'gum': 'Pirap'}, {'es': 'Desayunar', 'gum': 'Lѳtsѳ pi muchip'}, {'es': 'Barrer', 'gum': 'Palunchip'}, {'es': 'Trabajar', 'gum': 'Lutѳyu kuallip'}, {'es': 'Dormir', 'gum': 'Kip'}, {'es': 'Descansar', 'gum': 'Israp'}, {'es': 'Caminar', 'gum': 'May wan ip'}, {'es': 'Cepillarse', 'gum': 'Trukul pillmap'}, {'es': 'Papá', 'gum': 'Mѳskay'}, {'es': 'Mamá', 'gum': 'Usri'}, {'es': 'Hermano', 'gum': 'Nunek'}, {'es': 'Hermana', 'gum': 'Nuchak'}, {'es': 'Abuelo', 'gum': 'Shura'}, {'es': 'Abuela', 'gum': 'Shur'}, {'es': 'Tía', 'gum': 'Kuchiusri'}, {'es': 'Tío', 'gum': 'Kuchi Mѳskay'}, {'es': 'Esposo', 'gum': 'Kek'}, {'es': 'Esposa', 'gum': 'Srai'}, {'es': 'Niño', 'gum': 'unѳ'}, {'es': 'La abuela se baña', 'gum': 'shurape piran'}, {'es': 'Mama Manuela La madre de los misak', 'gum': 'Mama Manuela tru misakmerai usri'}, {'es': 'El niño desayuna', 'gum': 'unѳ lѳtsѳ mamik man'}, {'es': 'La tia trabaja', 'gum'

## NOTA:
Vamos a cargar un archivo JSON con algunas traducciones español - namuy wam 
Con los datos obtenidos procedemos a generar una conversación de contexto con el LLM

In [40]:

memory_chain = ConversationBufferMemory(
    return_messages=True,
    ai_prefix="Assistant",
    human_prefix="Human"
)

for translation in translations:
    memory_chain.save_context({"input": translation['es']}, {"output": translation['gum']})
memory_chain.load_memory_variables({})

{'history': [HumanMessage(content='Levantar'),
  AIMessage(content='Kutrap'),
  HumanMessage(content='Levantarse'),
  AIMessage(content='Kia kutrap'),
  HumanMessage(content='Bañar'),
  AIMessage(content='Pirap'),
  HumanMessage(content='Desayunar'),
  AIMessage(content='Lѳtsѳ pi muchip'),
  HumanMessage(content='Barrer'),
  AIMessage(content='Palunchip'),
  HumanMessage(content='Trabajar'),
  AIMessage(content='Lutѳyu kuallip'),
  HumanMessage(content='Dormir'),
  AIMessage(content='Kip'),
  HumanMessage(content='Descansar'),
  AIMessage(content='Israp'),
  HumanMessage(content='Caminar'),
  AIMessage(content='May wan ip'),
  HumanMessage(content='Cepillarse'),
  AIMessage(content='Trukul pillmap'),
  HumanMessage(content='Papá'),
  AIMessage(content='Mѳskay'),
  HumanMessage(content='Mamá'),
  AIMessage(content='Usri'),
  HumanMessage(content='Hermano'),
  AIMessage(content='Nunek'),
  HumanMessage(content='Hermana'),
  AIMessage(content='Nuchak'),
  HumanMessage(content='Abuelo'),
 

In [41]:

template = """
    The following is a friendly conversation between a human and an AI. The AI is an expert translator of Namuy Wam. If the AI does not know the answer to a question, it truthfully says it does not know.
    Current conversation:
    {history}
    Human: {input}
    Assistant:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation= ConversationChain(
    llm=llm_claude, 
    verbose=True, 
    memory=memory_chain,
    prompt=PROMPT
)

In [45]:
conversation.predict(input="Traduce 'El abuelo desayuna' a Namuy Wam")



> Entering new ConversationChain chain...
Prompt after formatting:

    The following is a friendly conversation between a human and an AI. The AI is an expert translator of Namuy Wam. If the AI does not know the answer to a question, it truthfully says it does not know.
    Current conversation:
    [HumanMessage(content='Levantar'), AIMessage(content='Kutrap'), HumanMessage(content='Levantarse'), AIMessage(content='Kia kutrap'), HumanMessage(content='Bañar'), AIMessage(content='Pirap'), HumanMessage(content='Desayunar'), AIMessage(content='Lѳtsѳ pi muchip'), HumanMessage(content='Barrer'), AIMessage(content='Palunchip'), HumanMessage(content='Trabajar'), AIMessage(content='Lutѳyu kuallip'), HumanMessage(content='Dormir'), AIMessage(content='Kip'), HumanMessage(content='Descansar'), AIMessage(content='Israp'), HumanMessage(content='Caminar'), AIMessage(content='May wan ip'), HumanMessage(content='Cepillarse'), AIMessage(content='Trukul pillmap'), HumanMessage(content='Papá'), AIMess

' La traducción de "El abuelo desayuna" al namuy wam es:\n\nShurpe lѳtsѳ pi muchip'